In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve

In [2]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

In [3]:
simp = True;

## Target function

we want to minimize the effective price, given a set of exponents for the buying
and selling asset each.

In [4]:
def deltaBySpot_(asset, s,v, b, w, j, e, Delta, a, min):
  f = (s - (v + b) * w) / w
  if asset == 'b':
    f = -f
  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
deltaBySpot['b']

b_b - s_b/w_b + v_b

In [5]:
deltaBySpot["s"];

-b_s + s_s/w_s - v_s

In [6]:
def spotByExp_(s, v, b, w, j, e, Delta, a, min):
  f = a * ((1 + 1/j) ** e)
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*((j_b + 1)/j_b)**e_b

In [7]:
spotByExp["s"];

a_s*((j_s + 1)/j_s)**e_s

In [8]:
def deltaByExp_(asset, s, v, b, w, j, e, Delta, a, min):
    f = deltaBySpot[asset].subs(s, spotByExp[asset])
    if simp:
        f = simplify(f)
    return f

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

-a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) + b_b + v_b

In [9]:
deltaByExp["s"];

a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - v_s

In [10]:
eff = Delta_s / Delta_b;
eff;

Delta_s/Delta_b

In [11]:
Delta_b_by_Exp_s = deltaByExp['s'] * spotByExp['b'] / spotByExp['s']
if simp:
    Delta_b_by_Exp_s = simplify(Delta_b_by_Exp_s)
Delta_b_by_Exp_s

a_b*((j_b + 1)/j_b)**e_b*(a_s*(j_s + 1)**e_s - j_s**e_s*w_s*(b_s + v_s))/(a_s*w_s*(j_s + 1)**e_s)

In [12]:
Delta_s_by_Exp_b = deltaByExp['b'] * spotByExp['s'] / spotByExp['b']
if simp:
    Delta_s_by_Exp_b = simplify(Delta_s_by_Exp_b)
Delta_s_by_Exp_b

a_s*((j_s + 1)/j_s)**e_s*(-a_b*(j_b + 1)**e_b + j_b**e_b*w_b*(b_b + v_b))/(a_b*w_b*(j_b + 1)**e_b)

In [13]:
effByExp_b = eff.subs({Delta_b: deltaByExp['b'], Delta_s: Delta_s_by_Exp_b})
if simp:
    effByExp_b = simplify(effByExp_b)
effByExp_b

a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s/a_b

In [14]:
effByExp_s = eff.subs({Delta_b: Delta_b_by_Exp_s, Delta_s: deltaByExp['s']})
if simp:
    effByExp_s = simplify(effByExp_s)
effByExp_s

a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s/a_b

In [15]:
simplify(effByExp_b - effByExp_s);

0

In [16]:
effByExps = effByExp_b;

In [17]:
H = hessian(effByExps, [e_b, e_s]);
H;

Matrix([
[                a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))**2/a_b, a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))*log((j_s + 1)/j_s)/a_b],
[a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))*log((j_s + 1)/j_s)/a_b,                 a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log((j_s + 1)/j_s)**2/a_b]])

In [18]:
topLeft = simplify(H[0, 0]);
topLeft;

a_s*(j_b/(j_b + 1))**e_b*((j_s + 1)/j_s)**e_s*log(j_b/(j_b + 1))**2/a_b

-> positive

In [19]:
det = H.det();
det;

0

## inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds
  given by our equation
- need to buy and sell minimum amounts
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not
  exceed expLimit (TODO)

In [20]:
def expBound_(asset, s, v, b, w, j, e, Delta, a, min):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = e - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - e # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

e_b - log((w_b*(b_b + v_b))**(1/log((j_b + 1)/j_b))/a_b**(1/log((j_b + 1)/j_b)))

In [21]:
expBound["s"];

-e_s + log((w_s*(b_s + v_s))**(1/log((j_s + 1)/j_s))/a_s**(1/log((j_s + 1)/j_s)))

In [22]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

def maxSpotBound_(asset, s, v, b, w, j, e, Delta, a, min):
  return spotByExp[asset] - I_max # <=! 0

maxSpotBound = {asset: maxSpotBound_(asset, *all_symbols[asset]) for asset in assets}
maxSpotBound['b']

-I_max + a_b*((j_b + 1)/j_b)**e_b

In [23]:
def minAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
    f = min - Delta # <=! 0
    f = f.subs(Delta, deltaByExp[asset])
    if simp:
        f = simplify(f)
    return f

minAmntBound = {asset: minAmntBound_(asset, *all_symbols[asset]) for asset in assets}
minAmntBound['b']

a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) - b_b + min_b - v_b

In [24]:
minAmntBound["s"];

-a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) + b_s + min_s + v_s

In [25]:
def maxAmntBound_(asset, s, v, b, w, j, e, Delta, a, min):
    if asset == 'b':
        max = b
    else:
        max = max_s
    f = Delta - max # <=! 0
    f = f.subs(Delta, deltaByExp[asset])
    if simp:
        f = simplify(f)
    return f

maxAmntBound = {asset: maxAmntBound_(asset, *all_symbols[asset]) for asset in assets}
maxAmntBound['b']

-a_b*(j_b + 1)**e_b/(j_b**e_b*w_b) + v_b

In [26]:
maxAmntBound["s"];

a_s*(j_s + 1)**e_s/(j_s**e_s*w_s) - b_s - max_s - v_s

## Lagrangian

In [27]:
# Decision variables
x1, x2 = e_b, e_s

# Constraints
g = [
    expBound['b'],
    expBound['s'],
    maxSpotBound['b'],
    minAmntBound['b'],
    maxAmntBound['b'],
    minAmntBound['s'],
    maxAmntBound['s'],
] 

# Lagrange multipliers for m constraints
lambdas = symbols('lambda1:{}'.format(len(g)+1), nonnegative=True)

# Objective function
f = effByExps

# Lagrangian
L = f + sum([lambdas[i] * g[i] for i in range(len(g))])
if simp:
    L = simplify(L)

# Differentiate Lagrangian with respect to decision variables
dL_dx1 = L.diff(x1)
dL_dx2 = L.diff(x2)
if simp:
    dL_dx1 = simplify(dL_dx1)
    dL_dx2 = simplify(dL_dx2)

# Equations from stationarity
equations = [Eq(dL_dx1, 0), Eq(dL_dx2, 0)]

# Complementary slackness conditions
for i in range(len(g)):
    equations.append(Eq(lambdas[i] * g[i], 0))

# Solve
solutions = nonlinsolve(equations, [x1, x2] + list(lambdas))
solutions

FiniteSet((log((a_b/I_max)**(1/log(j_b/(j_b + 1)))), log((a_s/(w_s*(b_s + v_s)))**(1/(log(j_s) - log(j_s + 1)))), 0, log((j_s + 1)**(a_s*j_b**log(a_b**(3/(log(j_b) - log(j_b + 1)))/I_max**(3/(log(j_b) - log(j_b + 1))))*w_s*(j_s*(j_s + 1))**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(w_s*(b_s + v_s))**(1/(log(j_s) - log(j_s + 1)))))/j_s**(a_s*j_b**log(a_b**(3/(log(j_b) - log(j_b + 1)))/I_max**(3/(log(j_b) - log(j_b + 1))))*w_s*(j_s*(j_s + 1))**log(a_s**(1/(log(j_s) - log(j_s + 1)))/(w_s*(b_s + v_s))**(1/(log(j_s) - log(j_s + 1))))))/(a_b*j_s**(2*log(a_s**(1/(log(j_s) - log(j_s + 1)))/(w_s*(b_s + v_s))**(1/(log(j_s) - log(j_s + 1)))))*w_s*(j_b**2*(j_b + 1))**log(a_b**(1/(log(j_b) - log(j_b + 1)))/I_max**(1/(log(j_b) - log(j_b + 1))))), Complement({a_s*j_b**log(a_b**(3/log(j_b/(j_b + 1)))/I_max**(3/log(j_b/(j_b + 1))))*j_s**log((w_s*(b_s + v_s))**(2/log(j_s/(j_s + 1)))/a_s**(2/log(j_s/(j_s + 1))))*(1/(j_b*(j_b + 1)**2))**log(a_b**(1/log(j_b/(j_b + 1))))*(j_b*(j_b**2 + 2*j_b + 1))**log(I_max**